In [2]:
# Cell 1 - imports & load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 80)
pd.set_option("display.width", 200)

# path to your merged client features CSV
path = r"E:\client_features.csv" 
df = pd.read_csv(path, sep=";", dtype=str)
print("Loaded rows:", len(df))


Loaded rows: 1908825


In [3]:
# Cell 2 - quick inspection 
print("Columns:\n", df.columns.tolist())
display(df.head(6))
display(df.info())


Columns:
 ['dailyLoanCount', 'promoLoanCount', 'debt', 'clientMaxAmount', 'subscriptionAmount', 'dueAmount', 'totalRefunded', 'clientID', 'subscriptionDate', 'unsubscriptionDate', 'penaltyDebt', 'loanAmount_sum', 'loanAmount_max', 'loanAmount_mean', 'amountPaid_sum', 'amountPaid_mean', 'penaltyAmount_sum', 'loanID_count', 'amount_sum', 'amount_mean', 'amount_count']


,dailyLoanCount,promoLoanCount,debt,clientMaxAmount,subscriptionAmount,dueAmount,totalRefunded,clientID,subscriptionDate,unsubscriptionDate,penaltyDebt,loanAmount_sum,loanAmount_max,loanAmount_mean,amountPaid_sum,amountPaid_mean,penaltyAmount_sum,loanID_count,amount_sum,amount_mean,amount_count
0,0,0,10411.0,25000.0,300.0,10750.0,1146.0,683755851,07/11/2023,2026-05-14 23:00:00.000,0.0,65333.0,33333.0,9333.285714285714,75333.0,10761.857142857143,35000.0,7.0,25312.0,2812.4444444444443,9
1,0,0,0.0,2000.0,300.0,0.0,0.0,671846141,07/11/2023,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0,0,0.0,20518.0,300.0,0.0,0.0,673587463,07/11/2023,2024-11-25 23:00:00.000,0.0,65500.0,31500.0,10916.666666666666,75560.0,12593.333333333334,30000.0,6.0,42372.0,10593.0,4
3,0,0,9988.0,8805.0,300.0,9460.0,182.0,681126794,07/11/2023,2024-11-07 23:00:00.000,5000.0,8800.0,8800.0,8800.0,10170.0,10170.0,5000.0,1.0,0.0,0.0,0
4,0,0,2418.0,9654.0,300.0,3225.0,1049.0,654758551,07/11/2023,2024-11-08 23:00:00.000,5000.0,7500.0,6000.0,2500.0,8598.0,2866.0,15000.0,3.0,7479.0,679.9090909090909,11
5,0,0,1075.0,1928.0,300.0,1075.0,0.0,651250572,07/11/2023,2025-12-24 23:00:00.000,0.0,5742.0,2121.0,1435.5,5444.0,1361.0,20000.0,4.0,0.0,0.0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1908825 entries, 0 to 1908824
Data columns (total 21 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   dailyLoanCount      object
 1   promoLoanCount      object
 2   debt                object
 3   clientMaxAmount     object
 4   subscriptionAmount  object
 5   dueAmount           object
 6   totalRefunded       object
 7   clientID            object
 8   subscriptionDate    object
 9   unsubscriptionDate  object
 10  penaltyDebt         object
 11  loanAmount_sum      object
 12  loanAmount_max      object
 13  loanAmount_mean     object
 14  amountPaid_sum      object
 15  amountPaid_mean     object
 16  penaltyAmount_sum   object
 17  loanID_count        object
 18  amount_sum          object
 19  amount_mean         object
 20  amount_count        object
dtypes: object(21)
memory usage: 305.8+ MB


None

In [4]:
# Cell 3 - convert numeric columns we need (robustly)
# list numeric cols we expect from earlier aggregation
num_cols = [
    "loanAmount_sum", "loanAmount_max", "loanAmount_mean",
    "amountPaid_sum", "amountPaid_mean",
    "penaltyAmount_sum", "loanID_count",
    "amount_sum", "amount_mean", "amount_count",
    "debt", "clientMaxAmount", "totalRefunded", "penaltyDebt"
]

# Convert if present; missing columns become 0
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c].str.replace(',', '').replace('', '0'), errors="coerce").fillna(0.0)
    else:
        df[c] = 0.0

# Ensure client id exists and is string
if "clientID" in df.columns:
    df["clientID"] = df["clientID"].astype(str)
elif "clientId" in df.columns:
    df["clientID"] = df["clientId"].astype(str)
elif "clientId" not in df.columns and "clientID" not in df.columns:
    raise RuntimeError("No clientID column found. Rename field to clientID/clientId.")

df = df.reset_index(drop=True)
print("Numeric conversion done. Sample:")
display(df[["clientID","loanAmount_sum","amountPaid_sum","debt","clientMaxAmount","totalRefunded","penaltyDebt","loanID_count",]].head())

Numeric conversion done. Sample:


,clientID,loanAmount_sum,amountPaid_sum,debt,clientMaxAmount,totalRefunded,penaltyDebt,loanID_count
0,683755851,65333.0,75333.0,10411.0,25000.0,1146.0,0.0,7.0
1,671846141,0.0,0.0,0.0,2000.0,0.0,0.0,0.0
2,673587463,65500.0,75560.0,0.0,20518.0,0.0,0.0,6.0
3,681126794,8800.0,10170.0,9988.0,8805.0,182.0,5000.0,1.0
4,654758551,7500.0,8598.0,2418.0,9654.0,1049.0,5000.0,3.0


In [4]:
#cell 4
# Drop columns flagged by supervisor
drop_cols = [
    "dailyLoanCount", "promoLoanCount", "debt",
    "subscriptionAmount", "dueAmount", "totalRefunded",
    "subscriptionDate", "unsubscriptionDate", "penaltyDebt"
]

df = df.drop(columns=[c for c in drop_cols if c in df.columns])

print("Columns after supervisor-approved drop:")
print(df.columns.tolist())


Columns after supervisor-approved drop:
['clientMaxAmount', 'clientID', 'loanAmount_sum', 'loanAmount_max', 'loanAmount_mean', 'amountPaid_sum', 'amountPaid_mean', 'penaltyAmount_sum', 'loanID_count', 'amount_sum', 'amount_mean', 'amount_count']


In [5]:
# Export only first 10 rows for review
df.head(10).to_csv(r"E:\client_features_sample.csv", index=False, sep=";")
print("Sample CSV saved successfully!")

Sample CSV saved successfully!


In [5]:

df_fe = df.copy()

# Feature 1: average loan size 
df_fe['avg_loan_size'] = df_fe['loanAmount_sum'] / df_fe['loanID_count'].clip(lower=1)

# Feature 2: transaction activity = how many refunds/transactions recorded
df_fe['tx_activity'] = df_fe['amount_count'].clip(lower=0)

# Feature 3: average transaction value
df_fe['avg_tx_value'] = df_fe['amount_sum'] / df_fe['amount_count'].clip(lower=1)

# Feature 4: client capacity proxy = ratio of max loan given to observed average loan
df_fe['capacity_ratio'] = df_fe['loanAmount_max'] / df_fe['avg_loan_size'].clip(lower=1)

# Clean NaN/inf
df_fe = df_fe.replace([np.inf, -np.inf], 0).fillna(0)

print("Engineered features added:")
print(df_fe[['clientID','avg_loan_size','tx_activity','avg_tx_value','capacity_ratio']].head(10))


Engineered features added:
    clientID  avg_loan_size  tx_activity  avg_tx_value  capacity_ratio
0  683755851    9333.285714            9   2812.444444        3.571411
1  671846141       0.000000            0      0.000000        0.000000
2  673587463   10916.666667            4  10593.000000        2.885496
3  681126794    8800.000000            0      0.000000        1.000000
4  654758551    2500.000000           11    679.909091        2.400000
5  651250572    1435.500000            0      0.000000        1.477534
6  650482240    3489.400000           13   2069.923077        2.006076
7  652987653    8750.000000            6   4748.500000        1.714286
8  677289119   13600.000000            4   3132.250000        2.205882
9  672454716   82500.000000            1   5500.000000        1.000000


In [7]:
# Export engineered features sample
df_fe.head(10).to_csv(r"E:\client_features_sample.csv", index=False, sep=";")
print("Sample CSV with engineered features saved successfully!")


Sample CSV with engineered features saved successfully!


In [6]:

def det_score(row):
    base = 500
    
    size_contrib     = min(2500, row['avg_loan_size'] * 0.2)
    activity_contrib = min(2500, row['tx_activity'] * 100)   # each tx adds some weight
    value_contrib    = min(2500, row['avg_tx_value'] * 0.05)
    capacity_contrib = min(2500, row['capacity_ratio'] * 500) # scaled ratio
    
    score = base + size_contrib + activity_contrib + value_contrib + capacity_contrib
    return min(10000, max(500, score))

df_fe['proxy_deterministic'] = df_fe.apply(det_score, axis=1)

print("Deterministic scoring sample:")
print(df_fe[['clientID','avg_loan_size','tx_activity','avg_tx_value','capacity_ratio','proxy_deterministic']].head(10))


Deterministic scoring sample:
    clientID  avg_loan_size  tx_activity  avg_tx_value  capacity_ratio  proxy_deterministic
0  683755851    9333.285714            9   2812.444444        3.571411          5192.984904
1  671846141       0.000000            0      0.000000        0.000000           500.000000
2  673587463   10916.666667            4  10593.000000        2.885496          5055.731425
3  681126794    8800.000000            0      0.000000        1.000000          2760.000000
4  654758551    2500.000000           11    679.909091        2.400000          3333.995455
5  651250572    1435.500000            0      0.000000        1.477534          1525.866980
6  650482240    3489.400000           13   2069.923077        2.006076          3604.413925
7  652987653    8750.000000            6   4748.500000        1.714286          3944.567857
8  677289119   13600.000000            4   3132.250000        2.205882          4659.553676
9  672454716   82500.000000            1   5500.00

In [7]:
df_fe = df_fe[df_fe['clientID'].notna() & (df_fe['clientID'] != "")]


In [10]:
# Optional: clip activity and tx_value at 99th percentile
for col in ["tx_activity", "avg_tx_value", "capacity_ratio"]:
    hi = df_fe[col].quantile(0.99)    df_fe[col] = df_fe[col].clip(upper=hi)


In [11]:
print("Score min:", df_fe['proxy_deterministic'].min())
print("Score max:", df_fe['proxy_deterministic'].max())
print("Any NaN clientIDs left?", df_fe['clientID'].isna().sum())


Score min: 500.0
Score max: 8462.874390243902
Any NaN clientIDs left? 0


In [12]:
# Export all scores
df_fe[['clientID','avg_loan_size','tx_activity','avg_tx_value','capacity_ratio','proxy_deterministic']] \
    .to_csv(r"E:\client_scores.csv", sep=";", index=False)

print("Scores exported to E:\\client_scores.csv")


Scores exported to E:\client_scores.csv


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Features for empirical model
X = df_fe[['repayment_ratio','loan_to_capacity','penalty_ratio']].fillna(0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Target: assume repayment_ratio as proxy for "good behavior"
y = df_fe['repayment_ratio'] * 10000

# Fit regression
reg = LinearRegression().fit(X_scaled, y)
df_fe['proxy_empirical'] = reg.predict(X_scaled)

# Scale to 500–10000
df_fe['proxy_empirical'] = 500 + (df_fe['proxy_empirical'] - df_fe['proxy_empirical'].min()) / \
    (df_fe['proxy_empirical'].max() - df_fe['proxy_empirical'].min()) * 9500

print("Empirical model coefficients:", dict(zip(['repayment_ratio','loan_to_capacity','penalty_ratio'], reg.coef_)))
print(df_fe[['clientID','proxy_empirical']].head(10))


Empirical model coefficients: {'repayment_ratio': np.float64(24261557.63953066), 'loan_to_capacity': np.float64(-2.7803550494705686e-09), 'penalty_ratio': np.float64(-1.1721410997037833e-07)}
    clientID  proxy_empirical
0  683755851       500.000000
1  671846141       500.000000
2  673587463       500.000000
3  681126794       500.000000
4  654758551       500.000000
5  651250572       500.000000
6  650482240       999.935490
7  652987653       916.660975
8  677289119       997.484119
9  672454716       500.000000


In [5]:
# Export only first 10 rows for review
df.head(10).to_csv(r"E:\client_features_sample.csv", index=False, sep=";")
print("Sample CSV saved successfully!")

Sample CSV saved successfully!


In [8]:
# Export to CSV for review
df.to_csv(r"E:\client_features.csv", index=False, sep=";")
print("CSV saved successfully!")

Sample CSV saved successfully!
